In [29]:
import os
import json
import glob

exp_dir = "logs/convbased_exp1"

In [ ]:
from preprocess import preprocess_training_set

In [23]:
from extract import run_pitch_extraction, run_embedding_extraction, generate_config, generate_filelist

f0_method = "rmvpe"
hop_length = 128
num_processes = 2
gpus = "0"
sample_rate = "48000"
embedder_model = "contentvec"
embedder_model_custom = None
include_mutes = 0

wav_path = os.path.join(exp_dir, "sliced_audios_16k")
os.makedirs(os.path.join(exp_dir, "f0"), exist_ok=True)
os.makedirs(os.path.join(exp_dir, "f0_voiced"), exist_ok=True)
os.makedirs(os.path.join(exp_dir, "extracted"), exist_ok=True)

chosen_embedder_model = (
    embedder_model_custom if embedder_model == "custom" else embedder_model
)
file_path = os.path.join(exp_dir, "model_info.json")
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
else:
    data = {}
data["embedder_model"] = chosen_embedder_model
with open(file_path, "w") as f:
    json.dump(data, f, indent=4)

files = []
for file in glob.glob(os.path.join(wav_path, "*.wav")):
    file_name = os.path.basename(file)
    file_info = [
        file,
        os.path.join(exp_dir, "f0", file_name + ".npy"),
        os.path.join(exp_dir, "f0_voiced", file_name + ".npy"),
        os.path.join(exp_dir, "extracted", file_name.replace("wav", "npy")),
    ]
    files.append(file_info)

devices = ["cpu"] if gpus == "-" else [f"cuda:{idx}" for idx in gpus.split("-")]


In [22]:
preprocess_training_set(
    input_root        = r"D:\Drives\OneDrive - NetDisk\Datasets 数据集\音频\开源数据集\merged_dataset\test",
    sr                = 48000,
    num_processes     = 32,
    exp_dir           = exp_dir,
    cut_preprocess    = "Automatic",
    process_effects   = True,
    noise_reduction   = False,
    reduction_strength= 0.7,
    chunk_len         = 3.0,
    overlap_len       = 0.3,
)

Starting preprocess with 32 processes...


100%|██████████| 59/59 [00:58<00:00,  1.02it/s]

Preprocess completed in 58.02 seconds on 05:31:43 seconds of audio.


In [24]:
run_pitch_extraction(files, devices, f0_method, hop_length, num_processes)

Starting pitch extraction on cuda:0 using rmvpe...
Pitch extraction completed in 104.77 seconds.


In [25]:
run_embedding_extraction(
    files, devices, embedder_model, embedder_model_custom, num_processes
)

Starting embedding extraction cores on cuda:0...
Embedding extraction completed in 37.49 seconds.


In [26]:
generate_config(sample_rate, exp_dir)

In [27]:
generate_filelist(exp_dir, sample_rate, include_mutes)

In [31]:
def analyze_speaker_distribution(filelist_path: str) -> tuple[list[int], list[int]]:
    """
    Analyze speaker ID distribution from filelist.txt
    
    Args:
        filelist_path (str): Path to the filelist.txt file
        
    Returns:
        tuple[list[int], list[int]]: A tuple containing:
            - List of sorted speaker IDs
            - List of missing speaker IDs
    """
    speakers = set()
    
    with open(filelist_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('|')
            if len(parts) > 1:
                speaker_id = parts[-1]
                if speaker_id.isdigit():
                    speakers.add(int(speaker_id))
    
    sorted_speakers = sorted(speakers)
    missing_speakers = []
    
    if sorted_speakers:
        for i in range(0, sorted_speakers[-1]):
            if i not in speakers:
                missing_speakers.append(i)
    
    return sorted_speakers, missing_speakers

# Main analysis
filelist_path = os.path.join(exp_dir, "filelist.txt")
sorted_speakers, missing_speakers = analyze_speaker_distribution(filelist_path)

# Log analysis results
print(f"Speaker ID distribution analysis:")
print(f"Total unique speakers: {len(sorted_speakers)}")
print(f"Speaker ID range: [{min(sorted_speakers) if sorted_speakers else 0}, {max(sorted_speakers) if sorted_speakers else 0}]")
print(f"Identified speaker IDs: {sorted_speakers}")

if missing_speakers:
    print(f"Missing speaker IDs: {missing_speakers}")
    print(f"Continuity status: Discontinuous (Gaps detected)")
else:
    print(f"Continuity status: Continuous")


Speaker ID distribution analysis:
Total unique speakers: 476
Speaker ID range: [0, 475]
Identified speaker IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 19